In [1]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv('songs_normalize.csv')

# İlk birkaç satırı inceleyelim
df.head()


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [2]:
# Eksik verileri kontrol et
print(df.isnull().sum())


artist              0
song                0
duration_ms         0
explicit            0
year                0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
genre               0
dtype: int64


In [3]:
import re
from nltk.corpus import stopwords

# Stop words (anlamsız kelimeler) için Türkçe listesi
stop_words = stopwords.words('turkish')

# Temizleme fonksiyonu
def temizle(metin):
    # Küçük harfe çevir
    metin = metin.lower()
    # Sayıları kaldır
    metin = re.sub(r'\d+', '', metin)
    # Noktalama işaretlerini kaldır
    metin = re.sub(r'[^\w\s]', '', metin)
    # Stop words'leri çıkar
    metin = " ".join([kelime for kelime in metin.split() if kelime not in stop_words])
    return metin

# Şarkı sözlerini temizle
df['cleaned_lyrics'] = df['song_lyrics'].apply(temizle)


KeyError: 'song_lyrics'

In [4]:
# Veri setindeki kolonları kontrol et
print(df.columns)


Index(['artist', 'song', 'duration_ms', 'explicit', 'year', 'popularity',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'genre'],
      dtype='object')


In [5]:
import pandas as pd

df = pd.read_csv("songs_normalize.csv")
df.head()


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [9]:
features = [
    'danceability', 'energy', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo'
]


In [10]:
X = df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(X_scaled)


In [12]:
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    if sarki_adi not in df['song'].values:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    index = df[df['song'] == sarki_adi].index[0]
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler


In [13]:
sarki_oner("Senorita", df, similarity_matrix)


'❌ Şarkı bulunamadı. Lütfen tam adını gir.'

In [14]:
df['song'].sample(10)


1636                              Needed Me
357          Pretty Green Eyes - Radio Edit
39                                Separated
1345    Waiting All Night (feat. Ella Eyre)
390                                  Unwell
582                               Let Me Go
486              The Closest Thing to Crazy
472                          American Idiot
889                             Violet Hill
1417                                   Maps
Name: song, dtype: object

In [17]:
sarki_oner("Needed Me", df, similarity_matrix)


["I'm Sprung - T-Pain",
 'Love - Keyshia Cole',
 'Say Yes - Floetry',
 "Don't - Bryson Tiller",
 "Don't - Bryson Tiller"]

In [18]:
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını küçük harfe çevir ve boşlukları temizle
    sarki_adi = sarki_adi.lower().strip()

    # Eğer şarkı veri setinde yoksa
    if sarki_adi not in df['song'].str.lower().str.strip().values:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    # Şarkıyı bul
    index = df[df['song'].str.lower().str.strip() == sarki_adi].index[0]
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler


In [19]:
sarki_oner("Senorita", df, similarity_matrix)


'❌ Şarkı bulunamadı. Lütfen tam adını gir.'

In [20]:
df[df['song'].str.contains('senorita', case=False, na=False)]


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre


In [21]:
def temizle_sarki_adi(sarki_adi):
    return re.sub(r'[^\w\s]', '', sarki_adi).strip().lower()

df['clean_song'] = df['song'].apply(temizle_sarki_adi)

# Şimdi 'clean_song' sütununu kullanarak sorgulama yapabiliriz
df[df['clean_song'].str.contains('senorita')]


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,clean_song


In [22]:
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    if sarki_adi not in df['clean_song'].values:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    index = df[df['clean_song'] == sarki_adi].index[0]
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler


In [23]:
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    # Temizlenmiş şarkı ismi ile veri setinde arama yap
    if sarki_adi not in df['clean_song'].values:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    # Şarkıyı bul
    index = df[df['clean_song'] == sarki_adi].index[0]
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler


In [24]:
sarki_oner("Senorita", df, similarity_matrix)


'❌ Şarkı bulunamadı. Lütfen tam adını gir.'

In [25]:
!pip install fuzzywuzzy



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from fuzzywuzzy import process

def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    # Fuzzy matching ile şarkıyı bul
    benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

    if benzer_sarkilar is None:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    en_iyi_eslesme = benzer_sarkilar[0]

    # Eşleşen şarkının index'ini al
    index = df[df['clean_song'] == en_iyi_eslesme].index[0]

    # Benzerlikleri hesapla
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler


C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [27]:
sarki_oner("Senorita", df, similarity_matrix)


['Blurred Lines - Robin Thicke',
 'Domino - Jessie J',
 'Somebody - Natalie La Rose',
 'Call Me Maybe - Carly Rae Jepsen',
 'Alors on danse - Radio Edit - Stromae']

In [28]:
!pip install python-Levenshtein


   ---------------------------------------- 0.0/100.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/100.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/100.7 kB ? eta -:--:--
   ------- ------------------------------- 20.5/100.7 kB 131.3 kB/s eta 0:00:01
   ------- ------------------------------- 20.5/100.7 kB 131.3 kB/s eta 0:00:01
   ------- ------------------------------- 20.5/100.7 kB 131.3 kB/s eta 0:00:01
   ------- ------------------------------- 20.5/100.7 kB 131.3 kB/s eta 0:00:01
   ------------ --------------------------- 30.7/100.7 kB 72.9 kB/s eta 0:00:01
   ------------ --------------------------- 30.7/100.7 kB 72.9 kB/s eta 0:00:01
   ------------------- ------------------- 51.2/100.7 kB 109.2 kB/s eta 0:00:01
   ----------------------- --------------- 61.4/100.7 kB 121.3 kB/s eta 0:00:01
   ----------------------- --------------- 61.4/100.7 kB 121.3 kB/s eta 0:00:01
   ----------------------- --------------- 61.4/100.7 kB 121.3 kB


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:12
   -------------------------------- ------- 1.3/1.6 MB 27.9 kB/s eta 0:00:1

In [30]:
sarki_oner("Shape of You", df, similarity_matrix)


['Options - NSG',
 'Because Of You - Ne-Yo',
 'Sunflower - Spider-Man: Into the Spider-Verse - Post Malone',
 'German - EO',
 'Sin Pijama - Becky G']

In [31]:
sarki_oner("Good For You", df, similarity_matrix)

['Lonely Together (feat. Rita Ora) - Avicii',
 'Just Can’t Get Enough - Black Eyed Peas',
 'Best Thing I Never Had - Beyoncé',
 'Crash and Burn - Savage Garden',
 'A New Day Has Come - Radio Remix - Céline Dion']

In [32]:
sarki_oner("Starboy", df, similarity_matrix)

['Vossi Bop - Stormzy',
 'Miss Independent - Kelly Clarkson',
 'Otro Trago - Sech',
 'Blame It - Jamie Foxx',
 'Dilemma - Nelly']

In [33]:
from fuzzywuzzy import process
import pandas as pd
import numpy as np

# Şarkı önerisi fonksiyonu
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    # Fuzzy matching ile şarkıyı bul
    benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

    if benzer_sarkilar is None:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    en_iyi_eslesme = benzer_sarkilar[0]

    # Eşleşen şarkının index'ini al
    index = df[df['clean_song'] == en_iyi_eslesme].index[0]

    # Benzerlikleri hesapla
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler

# Örneğin Mahşer için öneri alalım:
sarki_oner("Mahşer", df, similarity_matrix)


['Here - Alessia Cara',
 "This One's for You (feat. Zara Larsson) (Official Song UEFA EURO 2016) - David Guetta",
 'Reload - Radio Edit - Sebastian Ingrosso',
 'My Sacrifice - Creed',
 'Years - ラジオ・エディット - Alesso']

In [34]:
!pip install langdetect


     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
      ------------------------------------ 20.5/981.5 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 30.7/981.5 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 30.7/981.5 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 30.7/981.5 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 51.2/981.5 kB 187.0 kB/s eta 0:00:05
     - ----------------------------------- 51.2/981.5 kB 187.0 kB/s eta 0:00:05
     - ----------------------------------- 51.2/981.5 kB 187.0 kB/s eta 0:00:05
     - ----------------------------------- 51.2/981.5 kB 187.0 kB/s eta 0:00:05
     - ----------------------------------- 51.2/981.5 kB 187.0 kB/s eta 0:00:05
     - ----------------------------------- 51.2/981.5 kB 187.0 kB/s eta 0:00:05
     --- --------------------------------- 81.9/981.5 kB 148.1 kB/s eta 0:00:07
     --- --------------------------------- 81.9


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
from langdetect import detect
from fuzzywuzzy import process

def sarki_oner_dil_tespitli(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adının dilini tespit et
    try:
        dil = detect(sarki_adi)
    except:
        return "❌ Dil tespiti yapılamadı. Lütfen geçerli bir şarkı adı girin."

    # Dilin Türkçe mi İngilizce mi olduğunu kontrol et
    if dil == 'tr':
        # Türkçe şarkılarla öneri yap
        df_turkce = df[df['song'].str.contains(r'[ıİğĞşŞüÜöÖçÇ]', regex=True, na=False)]  # Türkçe karakter içeren şarkılar
        return sarki_oner(sarki_adi, df_turkce, benzerlik_matrisi)
    elif dil == 'en':
        # İngilizce şarkılarla öneri yap
        df_inglesiz = df[df['song'].str.contains(r'[a-zA-Z]', regex=True, na=False)]  # İngilizce karakter içeren şarkılar
        return sarki_oner(sarki_adi, df_inglesiz, benzerlik_matrisi)
    else:
        return "❌ Dil tespiti yapılamadı veya şarkı adı bilinmiyor."

# Test için bir şarkı adı yazın
sarki_oner_dil_tespitli("Mahşer", df, similarity_matrix)  # Türkçe bir şarkı


IndexError: single positional indexer is out-of-bounds

In [38]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Türkçe ve İngilizce şarkılar için fonksiyonlar
def get_benzerlik_matrisi(df):
    # Şarkı sözlerini vektörleştir
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['clean_song'])
    # Cosine similarity matrisini hesapla
    benzerlik_matrisi = cosine_similarity(tfidf_matrix)
    return benzerlik_matrisi

def sarki_oner_dil_tespitli(sarki_adi, df, df_turkce, df_inglesiz):
    # Şarkı adının dilini tespit et
    try:
        dil = detect(sarki_adi)
    except:
        return "❌ Dil tespiti yapılamadı. Lütfen geçerli bir şarkı adı girin."

    # Dilin Türkçe mi İngilizce mi olduğunu kontrol et
    if dil == 'tr':
        # Türkçe şarkılarla öneri yap
        benzerlik_matrisi_turkce = get_benzerlik_matrisi(df_turkce)
        return sarki_oner(sarki_adi, df_turkce, benzerlik_matrisi_turkce)
    elif dil == 'en':
        # İngilizce şarkılarla öneri yap
        benzerlik_matrisi_inglesiz = get_benzerlik_matrisi(df_inglesiz)
        return sarki_oner(sarki_adi, df_inglesiz, benzerlik_matrisi_inglesiz)
    else:
        return "❌ Dil tespiti yapılamadı veya şarkı adı bilinmiyor."


In [39]:
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    # Fuzzy matching ile şarkıyı bul
    benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

    if benzer_sarkilar is None:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    en_iyi_eslesme = benzer_sarkilar[0]

    # Eşleşen şarkının index'ini al
    index = df[df['clean_song'] == en_iyi_eslesme].index[0]

    # Benzerlikleri hesapla
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler


In [40]:
from fuzzywuzzy import process
from langdetect import detect

# Şarkı adı temizleme fonksiyonu
def temizle_sarki_adi(sarki_adi):
    sarki_adi = sarki_adi.lower()  # Küçük harfe çevir
    sarki_adi = re.sub(r'[^\w\s]', '', sarki_adi)  # Noktalama işaretlerini kaldır
    return sarki_adi

# Türkçe ve İngilizce şarkılarla öneri yapan fonksiyon
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    # Fuzzy matching ile şarkıyı bul
    benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

    if benzer_sarkilar is None:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    en_iyi_eslesme = benzer_sarkilar[0]

    # Eşleşen şarkının index'ini al
    index = df[df['clean_song'] == en_iyi_eslesme].index[0]

    # Benzerlikleri hesapla
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler

# Dil tespiti ve öneri fonksiyonu
def sarki_oner_dil_tespitli(sarki_adi, df, df_turkce, df_inglesiz):
    try:
        dil = detect(sarki_adi)
    except:
        return "❌ Dil tespiti yapılamadı. Lütfen geçerli bir şarkı adı girin."

    # Dilin Türkçe mi İngilizce mi olduğunu kontrol et
    if dil == 'tr':
        # Türkçe şarkılarla öneri yap
        df_turkce_benzerlik_matrisi = get_benzerlik_matrisi(df_turkce)
        return sarki_oner(sarki_adi, df_turkce, df_turkce_benzerlik_matrisi)
    elif dil == 'en':
        # İngilizce şarkılarla öneri yap
        df_inglesiz_benzerlik_matrisi = get_benzerlik_matrisi(df_inglesiz)
        return sarki_oner(sarki_adi, df_inglesiz, df_inglesiz_benzerlik_matrisi)
    else:
        return "❌ Dil tespiti yapılamadı veya şarkı adı bilinmiyor."

# Benzerlik matrisini hesaplamak
def get_benzerlik_matrisi(df):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['clean_song'])
    benzerlik_matrisi = cosine_similarity(tfidf_matrix)
    return benzerlik_matrisi

# Türkçe ve İngilizce şarkıları filtrelemek
df_turkce = df[df['song'].str.contains(r'[ıİğĞşŞüÜöÖçÇ]', regex=True, na=False)]  # Türkçe karakter içerenler
df_inglesiz = df[df['song'].str.contains(r'[a-zA-Z]', regex=True, na=False)]  # İngilizce karakter içerenler

# Şarkı önerisini test etmek
sarki_oner_dil_tespitli("Mahşer", df, df_turkce, df_inglesiz)  # Türkçe bir şarkı
sarki_oner_dil_tespitli("Shape of You", df, df_turkce, df_inglesiz)  # İngilizce bir şarkı


IndexError: index 1584 is out of bounds for axis 0 with size 1

In [41]:
from fuzzywuzzy import process
from langdetect import detect
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Şarkı adı temizleme fonksiyonu
def temizle_sarki_adi(sarki_adi):
    sarki_adi = sarki_adi.lower()  # Küçük harfe çevir
    sarki_adi = re.sub(r'[^\w\s]', '', sarki_adi)  # Noktalama işaretlerini kaldır
    return sarki_adi

# Türkçe ve İngilizce şarkılarla öneri yapan fonksiyon
def sarki_oner(sarki_adi, df, benzerlik_matrisi):
    # Şarkı adını temizle
    sarki_adi = temizle_sarki_adi(sarki_adi)

    # Fuzzy matching ile şarkıyı bul
    benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

    if benzer_sarkilar is None:
        return "❌ Şarkı bulunamadı. Lütfen tam adını gir."

    en_iyi_eslesme = benzer_sarkilar[0]

    # Eşleşen şarkının index'ini al
    try:
        index = df[df['clean_song'] == en_iyi_eslesme].index[0]
    except IndexError:
        return "❌ Şarkı adı eşleşmedi."

    # Benzerlikleri hesapla
    benzerlikler = list(enumerate(benzerlik_matrisi[index]))
    benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]

    # Önerilen şarkıları listele
    öneriler = []
    for i in benzerlikler:
        sarki = df.iloc[i[0]]['song']
        sanatci = df.iloc[i[0]]['artist']
        öneriler.append(f"{sarki} - {sanatci}")

    return öneriler

# Dil tespiti ve öneri fonksiyonu
def sarki_oner_dil_tespitli(sarki_adi, df, df_turkce, df_inglesiz):
    try:
        dil = detect(sarki_adi)
    except:
        return "❌ Dil tespiti yapılamadı. Lütfen geçerli bir şarkı adı girin."

    # Dilin Türkçe mi İngilizce mi olduğunu kontrol et
    if dil == 'tr':
        # Türkçe şarkılarla öneri yap
        df_turkce_benzerlik_matrisi = get_benzerlik_matrisi(df_turkce)
        return sarki_oner(sarki_adi, df_turkce, df_turkce_benzerlik_matrisi)
    elif dil == 'en':
        # İngilizce şarkılarla öneri yap
        df_inglesiz_benzerlik_matrisi = get_benzerlik_matrisi(df_inglesiz)
        return sarki_oner(sarki_adi, df_inglesiz, df_inglesiz_benzerlik_matrisi)
    else:
        return "❌ Dil tespiti yapılamadı veya şarkı adı bilinmiyor."

#


In [42]:
df['clean_song'] = df['song'].apply(temizle_sarki_adi)


In [43]:
benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)
print(benzer_sarkilar)  # Burada benzer_sarkilar'a bakarak sonucu görebilirsiniz


NameError: name 'sarki_adi' is not defined

In [44]:
# Şarkı adını belirleyin
sarki_adi = "Shape of You"  # Ya da "Mahşer" gibi bir şarkı adı yazabilirsiniz.

# Fuzzy matching ile şarkıyı bul
benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

# Sonucu yazdırın
print(benzer_sarkilar)  # Burada benzer_sarkilar'a bakarak sonucu görebilirsiniz


('shape of you', 100)


In [45]:
# Şarkı adını belirleyin
sarki_adi = "Shape of You"  # Ya da "Mahşer" gibi bir şarkı adı yazabilirsiniz.

# Fuzzy matching ile şarkıyı bul
benzer_sarkilar = process.extractOne(sarki_adi, df['clean_song'].values)

# Eşleşen şarkının adını al
en_iyi_eslesme = benzer_sarkilar[0]

# Eşleşen şarkının index'ini al
index = df[df['clean_song'] == en_iyi_eslesme].index[0]

# Benzerlikleri hesapla
benzerlikler = list(enumerate(similarity_matrix[index]))
benzerlikler = sorted(benzerlikler, key=lambda x: x[1], reverse=True)[1:6]  # 5 benzer şarkı

# Önerilen şarkıları listele
oneriler = []
for i in benzerlikler:
    sarki = df.iloc[i[0]]['song']
    sanatci = df.iloc[i[0]]['artist']
    oneriler.append(f"{sarki} - {sanatci}")

# Sonuçları yazdır
print("Önerilen şarkılar:")
for oner in oneriler:
    print(oner)


Önerilen şarkılar:
Options - NSG
Because Of You - Ne-Yo
Sunflower - Spider-Man: Into the Spider-Verse - Post Malone
German - EO
Sin Pijama - Becky G


In [49]:
# Test etmek için şarkı adı verelim
sarki_adi = "Shape of You"  # Örnek İngilizce şarkı
print(sarki_oner(sarki_adi, df, similarity_matrix))


['Options - NSG', 'Because Of You - Ne-Yo', 'Sunflower - Spider-Man: Into the Spider-Verse - Post Malone', 'German - EO', 'Sin Pijama - Becky G']
